# Ch4: Mathematics of Cryptography: Algebraic Structures
## 4.1 Permutation Group
In the permutation group we have $n$ elements that have been permutated (can be permuatted) in $n!$ ways. e.g. we have 

$$
P_1 = 
\begin{pmatrix}
1 & 2 & 3 \\
3 & 1 & 2
\end{pmatrix}
$$

We also have another permutation $P_2$ as:
$$
P_2 = 
\begin{pmatrix}
1 & 2 & 3 \\
1 & 3 & 2
\end{pmatrix}
$$

We can also compose permutations which will again give us a result in the Permutations group.

$$
P_3 = P_2 \circ P_1 = 
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}
$$

> Note: The permutation group is a non-abelian group as closure property isn't satisfied.

In [1]:
class Permutation(object):
    def __init__(self, permutation: list):
        self.permutation = permutation
        self.permutation_map = self.get_permutations_map(permutation)

    def value(self, index: int) -> int:
        return self.permutation[index]

    def permutate(self, sequence: list) -> list:
        result = [0] * len(sequence)
        for index, value in enumerate(sequence):
            result[self.permutation_map[index] - 1] = value
        return result

    def get_permutations_map(self, permutation: list) -> list:
        result = [0] * len(permutation)
        for index, value in enumerate(permutation):
            result[value - 1] = index + 1
        return result

    def compose(self, p):
        permutations = [0] * len(self.permutation)
        for index, value in enumerate(self.permutation):
            permutations[index] = p.permutation[value - 1]
        return Permutation(permutations)

In [2]:
p1 = Permutation([3, 1, 2])
p2 = Permutation([1, 3, 2])
print(p2.permutate([-4, 89, 17]))

[-4, 17, 89]


In [4]:
# We can also compose different permutation groups to get a new permutation group
p3 = p2.compose(p1)
print('Newly Created Permutation:', p3.permutation)
print(p3.permutate([-4, 89, 17]))

Newly Created Permutation: [3, 2, 1]
[17, 89, -4]


## 4.2 The Finite Galois Field ($GF(2^n)$) 
we represent the Galois Feld over $2^n$ as a Polynomial and define normal polynomial operations such as addition, subtractionn, multiplication, division over these feilds. We also define XOR operation which is the normal additive action ovr the Galois Feild and multiplication with modulo.

In [5]:
class Polynomial:
    def __init__(self, coefficients: dict):
        self.coefficients = coefficients

    def __repr__(self):
        representation = 'Polynomial{'
        for degree, coefficient in self.coefficients.items():
            representation += str(coefficient if coefficient != 1 else '') + 'x^' + str(degree) + ' '
        return representation + '}'

    def __add__(self, other):
        result = {}
        self.update_coefficients(other, result)
        self.update_coefficients(self, result)
        result = Polynomial(result)
        result.prune()
        return result

    def __sub__(self, other):
        result = {}
        self.update_coefficients(self, result)
        self.update_coefficients(other, result, factor=-1)
        result = Polynomial(result)
        result.prune()
        return result

    def __mul__(self, other):
        result = {}
        for degree1, coefficient1 in self.coefficients.items():
            for degree2, coefficient2 in other.coefficients.items():
                result[degree1 + degree2] = result.get(degree1 + degree2, 0) + coefficient1 * coefficient2
        result = Polynomial(result)
        result.prune()
        return result

    def __truediv__(self, divisor):
        dividend = Polynomial(self.coefficients.copy())
        quotient = {}
        while dividend.degree() >= divisor.degree():
            quotient_pow = dividend.degree() - divisor.degree()
            quotient_coefficient = dividend.coefficients[dividend.degree()] / divisor.coefficients[divisor.degree()]
            quotient[quotient_pow] = quotient.get(quotient_pow, 0) + quotient_coefficient
            dividend = dividend - (divisor * Polynomial({quotient_pow: quotient_coefficient}))
        return Polynomial(quotient), dividend

    def __mod__(self, divisor):
        return (self / divisor)[1]

    def __xor__(self, other):
        result = {}
        for degree, coefficient in self.coefficients.items():
            result[degree] = coefficient ^ other.coefficients.get(degree, 0)
        for degree, coefficient in other.coefficients.items():
            result[degree] = coefficient ^ self.coefficients.get(degree, 0)
        result = Polynomial(result)
        result.prune()
        return result

    def __neg__(self):
        result = Polynomial(self.coefficients.copy())
        for degree, coefficient in result.coefficients.items():
            result.coefficients[degree] = -coefficient
        return result

    def degree(self) -> int:
        return max(degree for degree in self.coefficients)

    def prune(self):
        zero_coefficient_degrees = set()
        for degree, coefficient in self.coefficients.items():
            if coefficient == 0:
                zero_coefficient_degrees.add(degree)
        for degree in zero_coefficient_degrees:
            del self.coefficients[degree]

    def additive_inverse(self):
        return -self

    @staticmethod
    def update_coefficients(polynomial, coefficients, factor=1):
        for degree, coefficient in polynomial.coefficients.items():
            coefficients[degree] = coefficients.get(degree, 0) + factor * coefficient

    @staticmethod
    def additive_identity():
        return Polynomial({})

In [6]:
# the polynomial is defined as a map of degree and corresponding coefficients
# Testing the adition operation
p1 = Polynomial({12: 1, 7: 1, 2: 1})
p2 = Polynomial({8: 1, 4: 1, 3: 1, 1: 1, 0: 1})
print(p1 + p2)

Polynomial{x^8 x^4 x^3 x^1 x^0 x^12 x^7 x^2 }


In [7]:
# testing subtraction
print(p1 - p2)

Polynomial{x^12 x^7 x^2 -1x^8 -1x^4 -1x^3 -1x^1 -1x^0 }


In [8]:
#testing multiplication
print(p1 * p2)

Polynomial{x^20 x^16 2x^15 x^13 x^12 x^11 2x^10 x^8 x^7 x^6 x^5 x^3 x^2 }


In [9]:
# testing negation
print(-p1)

Polynomial{-1x^12 -1x^7 -1x^2 }


In [10]:
# testing division
quotient, remainder = p1 / p2
print(quotient * p2 + remainder)
print(p1)

Polynomial{x^2 x^12 x^7 }
Polynomial{x^12 x^7 x^2 }


In [11]:
# testing the XOR operator
p3 = Polynomial({5: 1, 2: 1, 1: 1})
p4 = Polynomial({3: 1, 2: 1, 0: 1})
print(p3 ^ p4)

Polynomial{x^5 x^1 x^3 x^0 }


The additve identity is the zero Polynomial of the Galois Feild and the additve inverse is the negative of the Polynomial. 

In [12]:
print(p1.additive_identity())
print(p1.additive_inverse())
print(p1 + p1.additive_inverse())

Polynomial{}
Polynomial{-1x^12 -1x^7 -1x^2 }
Polynomial{}
